In [1]:
"""
The first step: to lift the data (tabular data at the end) from one of these text files. 
e.g. https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/bhattacharya2018/bhattacharya2018jpc56.txt
"""

"""
Couple of things to make progress:
1. 
1) For the table lifting, remove the ‘#’ in front
2) differentiate between the two types of table: the chronological information and the paleo information.
   -> choronological if the column name of the table has 'age'

"""   


import requests
import pandas as pd
import numpy as np

def getTable (url):
    txt_data = requests.get(url).text
    txt_data = txt_data.replace('\r', '')  # remove "\r" in text
    splited_data = txt_data.split('\n')   
    length = len(splited_data)
    
    # remove the ‘#’ in front
    for i in range(length):
        if splited_data[i].startswith("#"):
            splited_data[i] = splited_data[i][1:].lstrip()          
   
  
    """
    tabular data: 1. tab seperated 2. the number of columns is the same in the table.     
    """
    
    # if tab seperated, get the number of tab seperated words (the number of columns)
    num_cols = []
    for i in range(length):
        if "\t" in splited_data[i]:
            splited_data[i] = splited_data[i].split('\t')
            num_cols.append(len(splited_data[i]))
        else:
            num_cols.append(0)
            
    # get start index of table with # of columns            
    start_index = []
    for i, num in enumerate(num_cols):
        if num > 2 and i != len(num_cols)-1 and num == num_cols[i+1] and num != num_cols[i-1] or num>2 and i == 0 and num ==num_cols[i+1]:
            start_index.append(i)
            
    # get tabular data    
    tables = []
    for i in start_index:            
        
        table = [splited_data[i]]
        num = len(splited_data[i])
        for j in range(i+1, len(splited_data)):            
            if len(splited_data[j]) == num:      
                
                table.append(splited_data[j])
            else:
                break
        tables.append(table)       
        
    # convert dataframe
    df_list = []
    for table in tables:
        if len(table) > 2: # # of raw > 2
            df = pd.DataFrame(np.array(table[1:]), columns=table[0])
            
            
            # differentiate types of table: chronological and paleo information.           
            flag = 0
            for col_name in table[0]:
                if 'age' in col_name:  # choronological if the column name has 'age'
                    flag = 1
                    break
            
            if flag == 1: # the column name has 'age'
                df = df.style.set_caption('Chronological Information')
            else:
                df = df.style.set_caption('Paleo Information')
                
            
            df_list.append(df)
            
    return df_list

In [2]:
url = "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/bhattacharya2018/bhattacharya2018jpc56.txt"

In [3]:
df_list = getTable (url)
print(len(df_list))

1


In [4]:
df_list[0]

,depth_cm,age_calBP,dDwax,dDwax-iv,d13Cwax,percentJAS,JASerror
0,512,9798.6,-147.94,-148.94,-26.429,77.587,7.8528
1,537,9899.6,-150.36,-151.41,-26.313,73.239,7.8871
2,593,10134,-147.32,-148.47,-26.717,76.986,8.1359
3,609,10197,-149.57,-150.73,-26.118,74.624,8.0159
4,658,10395,-148.86,-150.08,-25.753,77.187,8.0041
5,701,10593,-150.65,-151.93,-26.282,72.452,7.8048
6,741,10789,-156.35,-157.68,-26.004,62.068,8.0316
7,762,10889,-152.29,-153.66,-26.508,68.636,7.8856
8,823,11189,-154.04,-155.67,-26.63,63.91,8.0646
9,876,11461,-148.75,-150.72,-26.35,74.31,8.0889


In [5]:
url1 = "https://www.ncei.noaa.gov/pub/data/paleo/contributions_by_author/khider2014/khider2014-raw.txt"

In [6]:
df_list1 = getTable (url1)

In [7]:
len(df_list1)

2

In [8]:
df_list1[0]

,Depth,Species,CAMS#,14CageÂ±1Ï,Calendar AgeÂ±1Ï
0,12cm,Gs,OS-36493,580Â±110,269Â±222
1,55cm,Gs/Gr,OS-37308,815Â±200,491Â±395
2,99cm,Gs/Gr,OS-37292,1010Â±100,688Â±205
3,145cm,Gs,OS-37306,1090Â±120,742Â±242
4,201cm,MP,90357,1750Â±15,1389Â±149
5,238cm,Gs,OS-36485,1900Â±100,1551Â±262
6,351cm,Gs/Gr,104528,2575Â±20,2368Â±232
7,402cm,Gs,OS-37289,3960Â±150,4069Â±433
8,501cm,Gs,OS-37303,4070Â±190,4245Â±530
9,610cm,Gs/Gr,104529,4925Â±30,5344Â±216


In [9]:
df_list1[1]

,depth_cm,Mg/Ca-g.rub-w,d18Og.rub-w,d13Cg.rub-w,d18Oc.mund,d13Cc.mund,notes
0,1,5.21,-999.90,-999.90,-999.90,-999.90,"data from Stott et al. (2004, 2007)"
1,1,-999.90,-999.90,-999.90,-0.30,2.45,
2,2.5,5.04,0.76,-2.83,-999.90,-999.90,
3,3,-999.90,1.53,-2.81,-999.90,-999.90,"data from Stott et al. (2004, 2007)"
4,4,-999.90,0.90,-2.78,-999.90,-999.90,
5,5,5.10,0.75,-2.64,0.03,2.77,
6,5,-999.90,0.95,-2.93,-999.90,-999.90,
7,6.5,5.04,0.74,-2.85,-999.90,-999.90,
8,7,5.01,0.92,-2.57,-999.90,-999.90,"data from Stott et al. (2004, 2007)"
9,7,-999.90,-999.90,-999.90,-0.27,2.67,


In [10]:
"""
The second step is to lift some of the metadata so it would be nice if you could learn how to use their API: 

https://www.ncdc.noaa.gov/paleo-search/api

If you use the Study ID from above, what can you retrieve?
"""

import re

# get metadata from extracted study ID
def getMetaData (url):
    txt_data = requests.get(url).text
    splited_data = txt_data.split('\n')
 
   
    # find "study" to get study id: 5 digits after "study"
    study_lines = []
    for line in splited_data:
        if "study" in line.lower():
            study_lines.append(line.lower().partition("study")[2])
            
    study_id = getStudyID(study_lines)
    
    api = "https://www.ncdc.noaa.gov/paleo-search/study/search.json?NOAAStudyId=" + study_id
    
    return requests.get(api).json()

            
def getStudyID(study_lines):            
    for line in study_lines:
        temp = re.search('\d{% s}'% 5, line)
        study_id = (temp.group(0) if temp else '')
        if study_id:
            return study_id        

In [11]:
getMetaData (url)

{'study': [{'xmlId': '23014',
   'NOAAStudyId': '24890',
   'studyName': 'Guaymas Basin 9-12kYrBP Leaf Wax Isotopes and Precipitation Reconstructions',
   'doi': None,
   'uuid': 'ec9a1fdf-f98e-4132-adf2-4243b25d4594',
   'dataPublisher': 'NOAA',
   'contactInfo': {'type': 'CONTACT INFORMATION',
    'shortName': 'DOC/NOAA/NESDIS/NCEI',
    'longName': 'National Centers for Environmental Information, NESDIS, NOAA, U.S. Department of Commerce ',
    'address': '325 Broadway, E/NE31',
    'city': 'Boulder',
    'state': 'CO',
    'postalCode': '80305-3328',
    'country': 'USA',
    'dataCenterUrl': 'https://www.ncdc.noaa.gov/data-access/paleoclimatology-data',
    'email': 'paleo@noaa.gov',
    'phone': '303-497-6280',
    'fax': '303-497-6513',
    'constraints': 'Please cite original publication, online resource, dataset and publication DOIs (where available), and date accessed when using downloaded data. If there is no publication information, please cite investigator, title, online r

In [12]:
getMetaData (url1)
            

{'study': [{'xmlId': '13818',
   'NOAAStudyId': '16055',
   'studyName': 'Western Tropical Pacific SST and Isotope Data From MD98-2181 During the Holocene',
   'doi': None,
   'uuid': '02ef0ad2-716a-4e3a-a0c3-db797d27ffe7',
   'dataPublisher': 'NOAA',
   'contactInfo': {'type': 'CONTACT INFORMATION',
    'shortName': 'DOC/NOAA/NESDIS/NCEI',
    'longName': 'National Centers for Environmental Information, NESDIS, NOAA, U.S. Department of Commerce ',
    'address': '325 Broadway, E/NE31',
    'city': 'Boulder',
    'state': 'CO',
    'postalCode': '80305-3328',
    'country': 'USA',
    'dataCenterUrl': 'https://www.ncdc.noaa.gov/data-access/paleoclimatology-data',
    'email': 'paleo@noaa.gov',
    'phone': '303-497-6280',
    'fax': '303-497-6513',
    'constraints': 'Please cite original publication, online resource, dataset and publication DOIs (where available), and date accessed when using downloaded data. If there is no publication information, please cite investigator, title, onl